In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm as cm
from scipy.stats import binned_statistic
from scipy.stats import sem
import random
import warnings
import plot_utils as plu

## LD 
Throughout the notebook, LD is measured by r^2 (the classical correlation coefficient)

In [ ]:
print("* Computing and plotting LD...")

#### Compute correlation between all pairs of SNPs for each generated/real dataset

In [ ]:
categ = infiles.keys()
hcor_snp = dict()
for i,cat in enumerate(categ):
    print(cat)
    with np.errstate(divide='ignore', invalid='ignore'): 
        # Catch warnings due to fixed sites in dataset (the correlation value will be np.nan for pairs involving these sites)
        hcor_snp[cat] = np.corrcoef(datasets[cat], rowvar=False)**2  # r2

### Plot LD or COV binned by distance between SNPs 
- LD is binned into 'nbins'
- that are cut on a logscale if logscale=True
- For regressions plot of non-binned LD, we randomly subsample 'nsamplesets' pairs for computation/visualization convenience


In [ ]:
_, region_len, snps_on_same_chrom = plu.get_dist(position_fname['Real'], region_len_only=True,  kept_preprocessing=keptsnpdic['Real'])

nbins=50
nsamplesets=10000
logscale=True
bins = nbins
binsPerDist = nbins
if logscale: binsPerDist = np.logspace(np.log(1), np.log(region_len), nbins)

In [ ]:
# Compute LD binned by distance
# For each category use all sites that are SNPs (full)
# (done by simply removing pairs for which LD is nan)
if not matching_SNPs:    

    binnedPerDistLD = dict()
    i = 1
    plt.figure(figsize=(10,len(hcor_snp)*5))
    for cat, mat in hcor_snp.items():
        flathcor = mat[np.triu_indices(mat.shape[0])]
        if cat=='Real':
            flatreal = flathcor
            isnanReal = np.isnan(flatreal)

        isnan = np.isnan(flathcor)
        curr_dist = plu.get_dist(position_fname[cat], kept_preprocessing=keptsnpdic[cat])[0]
        print(f"{cat}: {np.sum(isnan)} nan correlations filtered out of {len(isnan)} pairs.")

        # For each dataset LD pairs are stratified by SNP distance and cut into 'nbins' bins
        ld = binned_statistic(curr_dist[~isnan], flathcor[~isnan], statistic = 'mean', bins=binsPerDist)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning) # so that empty bins do not raise a warning
            binnedPerDistLD[cat] = pd.DataFrame({'bin_edges':ld.bin_edges[:-1],
                                          'LD': ld.statistic,
                                          #'sd': binned_statistic(curr_dist[~isnan], flathcor[~isnan], statistic = 'std', bins=binsPerDist).statistic,
                                          'sem': binned_statistic(curr_dist[~isnan], flathcor[~isnan], statistic = sem, bins=binsPerDist).statistic,
                                          'cat': cat, 'logscale': logscale})

        # Plotting quantiles ?
        plu.plotregquant(x=flatreal, y=flathcor, 
                         keys=['Real',cat], statname='LD', col=colpal[cat], 
                         step=0.05,
                         ax= plt.subplot(len(hcor_snp),2,i))
        i+=1
        plt.title(f'Quantiles LD {cat} vs Real')
        # removing nan values and subsampling before doing the regression to have a reasonnable number of points
        if matching_SNPs:
            isnanInter = isnanReal | isnan
            keepforplotreg = random.sample(list(np.where(~isnanInter)[0]), nsamplesets)
            plu.plotreg(x=flatreal[keepforplotreg], y=flathcor[keepforplotreg], 
                        keys=['Real',cat], statname='LD', col=colpal[cat], 
                        ax= plt.subplot(len(hcor_snp),2,i))
            plt.title(f'LD {cat} vs Real')
        i+=1
    plt.savefig(outDir + "LD_generated_vs_real_fullSNP.pdf")

In [ ]:
# Plot LD as a fonction of binned distances  
# except when SNPs are spread accross different chromosomes
if (not matching_SNPs) and snps_on_same_chrom: #position_fname['Real'] != "1kg_real/805snps.legend":
    plt.figure(figsize=(5,5))
    for cat, bld in binnedPerDistLD.items():
        plt.errorbar(bld.bin_edges.values, bld.LD.values, bld['sem'].values, label=cat, alpha=.65,linewidth=3)
    plt.title("Binned LD +/- 1 sem")
    if (logscale): plt.xscale('log')   
    #plt.yscale('log')
    plt.xlabel("Distance between SNPs (bp) [Left bound of distance bin]")
    plt.ylabel("Average LD in bin")
    plt.legend()
    plt.savefig(outDir + "correlation_vs_dist_fullSNP.pdf")

In [ ]:
# Compute LD binned by distance
# Take only sites that are SNPs in all datasets (intersect)
# (eg intersection of SNPs in Real, SNPs in GAN, SNPs in RBM etc)
# -> Makes sense only if there is a correspondence between sites

if matching_SNPs:    
    binnedLD = dict()
    binnedPerDistLD = dict()
    kept_snp = ~is_fixed
    n_kept_snp = np.sum(kept_snp)
    realdist = plu.get_dist(position_fname['Real'], kept_preprocessing=keptsnpdic['Real'], kept_snp=kept_snp)[0]
    mat = hcor_snp['Real']
    # filter and flatten
    flatreal = (mat[np.ix_(kept_snp,kept_snp)])[np.triu_indices(n_kept_snp)]
    isnanReal = np.isnan(flatreal)
    i=1
    plt.figure(figsize=(10,len(hcor_snp)*5))

    for cat, mat in hcor_snp.items():
        flathcor = (mat[np.ix_(kept_snp,kept_snp)])[np.triu_indices(n_kept_snp)]
        isnan = np.isnan(flathcor)
        if position_fname[cat] != position_fname['Real']:
            curr_dist = plu.get_dist(position_fname[cat],  kept_preprocessing=keptsnpdic[cat], kept_snp=kept_snp)[0]
        else:
            curr_dist = realdist 

        # For each dataset LD pairs are stratified by SNP distance and cut into 'nbins' bins
        # bin per SNP distance
        ld = binned_statistic(curr_dist[~isnan], flathcor[~isnan], statistic = 'mean', bins=binsPerDist)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning) # so that empty bins do not raise a warning
            binnedPerDistLD[cat] = pd.DataFrame({'bin_edges':ld.bin_edges[:-1],
                                          'LD': ld.statistic,
                                          #'sd': binned_statistic(curr_dist[~isnan], flathcor[~isnan], statistic = 'std', bins=binsPerDist).statistic,
                                          'sem': binned_statistic(curr_dist[~isnan], flathcor[~isnan], statistic = sem, bins=binsPerDist).statistic,
                                          'cat': cat, 'logscale': logscale})

        # For each dataset LD pairs are stratified by LD values in Real and cut into 'nbins' bins
        # binnedLD contains the average, std of LD values in each bin
        isnan = np.isnan(flathcor) |  np.isnan(flatreal) 
        ld = binned_statistic(flatreal[~isnan], flathcor[~isnan], statistic = 'mean', bins=bins)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning) # so that empty bins do not raise a warning
            binnedLD[cat] = pd.DataFrame({'bin_edges':ld.bin_edges[:-1],
                                          'LD': ld.statistic,
                                          'sd': binned_statistic(flatreal[~isnan], flathcor[~isnan], statistic = 'std', bins=bins).statistic,
                                          'sem': binned_statistic(flatreal[~isnan], flathcor[~isnan], statistic = sem, bins=bins).statistic,
                                          'cat': cat, 'logscale': logscale})

        # Plotting quantiles ?
        plu.plotregquant(x=flatreal, y=flathcor, 
                         keys=['Real',cat], statname='LD', col=colpal[cat], 
                         step=0.05,
                         ax= plt.subplot(len(hcor_snp),2,i))
        i+=1
        plt.title(f'Quantiles LD {cat} vs Real')

        if matching_SNPs:
            # removing nan values and subsampling before doing the regression to have a reasonnable number of points
            isnanInter = isnanReal | isnan
            keepforplotreg = random.sample(list(np.where(~isnanInter)[0]), nsamplesets)
            plu.plotreg(x=flatreal[keepforplotreg], y=flathcor[keepforplotreg], 
                        keys=['Real',cat], statname='LD', col=colpal[cat], 
                        ax= plt.subplot(len(hcor_snp),2,i))
            i+=1
            plt.title(f'LD {cat} vs Real')
    plt.savefig(outDir + "LD_generated_vs_real_intersectSNP.pdf")

In [ ]:
# Plot LD as a fonction of binned distances
# except when SNPs are spread accross different chromosomes
if matching_SNPs and snps_on_same_chrom: #(position_fname['Real']!="1kg_real/805snps.legend"):
    plt.figure(figsize=(5,5))
    for cat, bld in binnedPerDistLD.items():
        plt.errorbar(bld.bin_edges.values, bld.LD.values, bld['sem'].values, label=cat, alpha=.65,linewidth=3)
    plt.title("Binned LD +/- 1 sem")
    if (logscale): plt.xscale('log')   
    #plt.yscale('log')
    plt.xlabel("Distance between SNPs (bp) [Left bound of distance bin]")
    plt.ylabel("Average LD in bin")
    plt.legend()
    plt.savefig(outDir + "correlation_vs_dist_intersectSNP.pdf")

In [ ]:
# For each dataset LD pairs were stratified by LD values in Real, cut into nbins bins
# binnedLD contains the average LD in each bin
# Plot generated average LD as a function of the real average LD in the bins
if matching_SNPs:
    plt.figure(figsize=(10,10))
    for cat, bld in binnedLD.items():
        plt.errorbar(bld.bin_edges.values, bld.LD.values, bld['sem'].values, label=cat, alpha=1, marker='o')
    plt.title("Binned LD +/- 1 sem")
    #if (logscale): plt.xscale('log')
    plt.xlabel("Bins (LD in Real)")
    plt.ylabel("Average LD in bin")
    plt.legend()
    plt.savefig(outDir+'LD_{}bins_{}fixedremoved.pdf'.format(nbins,'logdist_' if logscale else ''))

#### Plotting LD (block) matrix
- LD is r2
- if mirror=True plot the symmetrical matrix for each category
- if mirror=False plot the Generated LD in upper triangle versus the Real LD in lower triangle
- if diff=True plot the difference between generated and true LD values, else plot regular LD values

In [ ]:
print("* Plotting LD block matrices...")

# Set edges of the region for which to plot LD block matrix (l=0, f='end') for full region
# not used as for now apart from the filename
l_bound=None
r_bound=None

In [ ]:

# mirror (bool): plot symmetrical matrix or generated vs real?
# diff (bool): plot LD values or generated minus real ?

for snpcode in ("fullSNP", "intersectSNP"):
    for (mirror, diff) in ((True,False),(True,True), (False,False)):
        if (not matching_SNPs) and (snpcode=="fullSNP") and (diff or not mirror):
            print(f'Warning: not plotting LD for {snpcode} mirror={mirror} diff={diff}',
                  ' because SNP have no one-to-one correspondence and matrices might have different sizes')
        elif (matching_SNPs) and (snpcode=="fullSNP"):
            pass
        else:
            outfilename = f"LD_HEATMAP_{snpcode}_bounds={l_bound}-{r_bound}_mirror={mirror}_diff={diff}.pdf" 
            #print(outfilename)
            fig = plt.figure(figsize=(10*len(hcor_snp),10))
            plu.plotLDblock(hcor_snp, 
                            left=l_bound, right=r_bound,  # None, None -> takes all SNPs
                            mirror=mirror, diff=diff,
                            is_fixed=is_fixed, is_fixed_dic=is_fixed_dic,
                            suptitle_kws={'t':outfilename}
                           )
            plt.title(outfilename)
            plt.savefig(outDir+outfilename)
            plt.show()


In [ ]:
print('****************************************************************\n*** Computation and plotting LD DONE. Figures saved in {} ***\n****************************************************************'.format(outDir))